# LogisticRegression

## data 불러오기

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data=pd.read_csv("breast-cancer-wisconsin.csv")
x=data[data.columns[1:10]]
y=data[['Class']]

In [2]:
data.head(10)

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0
5,1017122,8,10,10,8,7,10,9,7,1,1
6,1018099,1,1,1,1,2,10,3,1,1,0
7,1018561,2,1,2,1,2,1,3,1,1,0
8,1033078,2,1,1,1,2,1,1,1,5,0
9,1033078,4,2,1,1,2,1,2,1,1,0


In [3]:
x

,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1
3,6,8,8,1,3,4,3,7,1
4,4,1,1,3,2,1,3,1,1
...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1
679,2,1,1,1,2,1,1,1,1
680,5,10,10,3,7,3,8,10,2
681,4,8,6,4,3,4,10,6,1


## data set 분리

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(x,y,stratify=y,random_state=42)

## MinMax Scaler

In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
scaler=MinMaxScaler()
scaler.fit(x_train)
x_scaled_train=scaler.transform(x_train)
x_scaled_test=scaler.transform(x_test)

## 기본 모델 적용

In [7]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()

In [9]:
model.fit(x_scaled_train,y_train)

LogisticRegression()

In [10]:
pred_train=model.predict(x_scaled_train)
model.score(x_scaled_train,y_train)

0.97265625

## 혼동행렬 확인

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
confusion_matrix=confusion_matrix(y_train,pred_train)
print(confusion_matrix)

[[328   5]
 [  9 170]]


In [13]:
from sklearn.metrics import classification_report

In [14]:
cfreport=classification_report(y_train,pred_train)
print(cfreport)

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       333
           1       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



## Test set 확인

In [15]:
pred_test=model.predict(x_scaled_test)
model.score(x_scaled_test,y_test)

0.9590643274853801

In [18]:
from sklearn.metrics import confusion_matrix

In [19]:
confusion_test=confusion_matrix(y_test,pred_test)
print(confusion_test)

[[106   5]
 [  2  58]]


In [20]:
cfreport=classification_report(y_test,pred_test)
print(cfreport)


              precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



## HyperParameter Tuning

### Grid Search

In [22]:
from sklearn.model_selection import GridSearchCV
param_grid={"C":[0.001,0.01,0.1,1,10,100]}
grid_search=GridSearchCV(LogisticRegression(),param_grid,cv=5,return_train_score=True)
# 파라미터 찾기
grid_search.fit(x_scaled_train,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
             return_train_score=True)

In [24]:
print("Best Parameter : {}".format(grid_search.best_params_))
print("Best Cross-validity Score : {:.4f}".format(grid_search.best_score_))
print('Test set Score : {:.4f}'.format(grid_search.score(x_scaled_test,y_test)))

Best Parameter : {'C': 10}
Best Cross-validity Score : 0.9726
Test set Score : 0.9591


### Random Search

In [26]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
param_distribs={"C":randint(low=0.001,high=100)} # 무작위 난수 생성
random_search=RandomizedSearchCV(LogisticRegression(),param_distributions=param_distribs,n_iter=100,cv=5,return_train_score=True)
# 파라미터 찾기

random_search.fit(x_scaled_train,y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000195EE426430>},
                   return_train_score=True)

In [27]:
print("Best Parameter : {}".format(random_search.best_params_))
print("Best Cross-validity Score : {:.4f}".format(random_search.best_score_))
print('Test set Score : {:.4f}'.format(random_search.score(x_scaled_test,y_test)))

Best Parameter : {'C': 11}
Best Cross-validity Score : 0.9745
Test set Score : 0.9591


## Parameter Tuning

In [30]:
model=LogisticRegression(C=11)

In [32]:
model.fit(x_scaled_train,y_train)

LogisticRegression(C=11)

In [33]:
pred_train=model.predict(x_scaled_train)
model.score(x_scaled_train,y_train)

0.9765625

In [34]:
cfreport=classification_report(y_train,pred_train)
print(cfreport)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       333
           1       0.97      0.96      0.97       179

    accuracy                           0.98       512
   macro avg       0.98      0.97      0.97       512
weighted avg       0.98      0.98      0.98       512



In [35]:
pred_test=model.predict(x_scaled_test)
model.score(x_scaled_test,y_test)

0.9590643274853801

In [36]:
cfreport=classification_report(y_test,pred_test)
print(cfreport)


              precision    recall  f1-score   support

           0       0.98      0.95      0.97       111
           1       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171

